Consideramos el modelo:

$$\mu = 25 - 5 \log_{10}(H_0/100) + 5\log_{10}(D_L/Mpc)$$

Donde $D_L$ tiene como solución aproximada:

$$D_L = {c \over H_0} (1 + z)[\eta(1, \Omega_m) - \eta(1/(1+z), \Omega_m)]$$

Siendo:

$$\eta(a,\Omega_m) = 2\sqrt{s^2+1}[a^{-4} - 0.1540sa^{-3} + 0.4304s^2a^{-2} + 0.19097s^3a^{-1} + 0.066941s^4]^{-1/8}$$

Que a su vez:

$$s^3 = (1- \Omega_m)/\Omega_m$$

Podemos hacer una estimación de los parámetros $H_0$ y $\Omega_m$ que mejor ajustan los datos de nuestro modelo utilizando la función Likelihood:

$$\mathcal{L} \propto \prod_i^n {1 \over 2\pi \sigma_i^2}exp \left( - \frac{(x_i-\mu)^2}{2\sigma_i^2}\right )$$

o bien:

$$-ln(\mathcal{L}) \propto {1 \over 2} \sum_i^n \left( \frac{(y_i - \lambda(x_i, \vec{\theta}))^2}{\sigma_i^2} \right)$$

Trataremos de maximizar esta función utilizando el algoritmo de Metrópolis